# Session X: Email Generation

Ideally, we want a set of email queries that we can answer with our dataset.
This is the code used to generate order status emails. 
You can adapt this code to make other types also.

In [2]:
# # Install pandas, openai if you dont already have it
!pip install pandas openai

## Load Data

In [28]:
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

data_folder = os.getcwd().split("genai_solution")[0] + "genai_solution\\data\\"

In [4]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_folder) if isfile(join(data_folder, f))]

In [5]:
onlyfiles

['.gitignore',
 'image.png',
 'new_retail_data.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'order_status_emails.csv',
 'product_category_name_translation.csv',
 'README.md']

In [6]:
orders = pd.read_csv(data_folder + 'olist_orders_dataset.csv')  
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [7]:
order_items = pd.read_csv(data_folder + 'olist_order_items_dataset.csv')  
products = pd.read_csv(data_folder + 'olist_products_dataset.csv')  
product_name_translation = pd.read_csv(data_folder + 'product_category_name_translation.csv')  

order_items = order_items.merge(products, "left", on="product_id").merge(product_name_translation,"left", on="product_category_name")

In [8]:
customers = pd.read_csv(data_folder + 'olist_customers_dataset.csv')  
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## Define LLM connection

In [9]:
from openai import OpenAI
# add your OPENAI_API_KEY here

def query_open_ai(prompt, temperature=0.7):
    client = OpenAI(api_key=OPENAI_API_KEY)
    completions = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": prompt
            }
        ],
        temperature=temperature
    )
 
    return completions.choices[0].message.content

I got the LLM to generate a list of potential email queries. I only order status is used in this example.

In [27]:
categories = {"Order-Related Issues":
["Order Status – Customers asking about the status of their orders (e.g., 'Where is my order?').",
"Order Modifications – Requests to change order details (e.g., address change, item swap).",
"Order Cancellations – Customers wanting to cancel an order before shipping."],
"Payment & Billing Issues":
["Payment Failures – Issues with processing payments.",
"Refund Requests – Customers requesting refunds for returned or undelivered items.",
"Incorrect Charges – Complaints about being overcharged or double charged."],
"Shipping & Delivery Issues":
["Delayed Delivery – Customers complaining about late shipments.",
"Lost or Missing Packages – Orders that haven’t arrived or are marked as delivered but not received.",
"Damaged Items – Complaints about receiving broken or defective products."],
"Returns & Exchanges":
["Return Requests – Customers asking how to return a product.",
"Exchange Requests – Requests to swap an item for another."],
"Product-Related Queries":
["Product Information – Questions about product specifications, availability, or compatibility.",
"Product Authenticity & Quality – Concerns about whether the product is genuine or meets expectations."],
"Technical Issues":
["Website or App Issues – Customers reporting glitches, login problems, or checkout failures.",
"Account Problems – Issues with logging in, password resets, or account security."],
"Seller-Related Complaints":
["Seller Responsiveness – Customers complaining about unresponsive sellers.",
"Seller Misrepresentation – Concerns about misleading product descriptions or pricing."],
"Promotions & Discounts":
["Promo Code Issues – Complaints about discount codes not working.",
"Price Adjustments – Requests for partial refunds due to price drops after purchase."],
"General Inquiries & Feedback":
["General Questions – Miscellaneous questions about policies, store operations, etc.",
"Customer Feedback & Complaints – Compliments, complaints, or improvement suggestions."]}

In [71]:
customers = customers.sample(frac=1).reset_index(drop=True)

## Order status Emails

I got the LLM to generate a list of names and emails for use in the email generation.

In [ ]:
customer_name_email = [
{ "name": "John Smith", "email": "john.smith@gmail.com" },
{ "name": "Jane Doe", "email": "jane.doe@yahoo.com" },
{ "name": "Alice Johnson", "email": "alice.johnson@outlook.com" },
{ "name": "Robert Brown", "email": "robert.brown@gmail.com" },
{ "name": "Emily Davis", "email": "emily.davis@hotmail.com" },
{ "name": "Michael Wilson", "email": "michael.wilson@yahoo.com" },
{ "name": "Sarah Martinez", "email": "sarah.martinez@gmail.com" },
{ "name": "David Anderson", "email": "david.anderson@icloud.com" },
{ "name": "Emma Thomas", "email": "emma.thomas@aol.com" },
{ "name": "James White", "email": "james.white@outlook.com" },
{ "name": "Olivia Harris", "email": "olivia.harris@gmail.com" },
{ "name": "William Martin", "email": "william.martin@yahoo.com" },
{ "name": "Sophia Thompson", "email": "sophia.thompson@hotmail.com" },
{ "name": "Benjamin Garcia", "email": "benjamin.garcia@icloud.com" },
{ "name": "Charlotte Martinez", "email": "charlotte.martinez@gmail.com" },
{ "name": "Daniel Robinson", "email": "daniel.robinson@outlook.com" },
{ "name": "Mia Clark", "email": "mia.clark@yahoo.com" },
{ "name": "Lucas Rodriguez", "email": "lucas.rodriguez@gmail.com" },
{ "name": "Amelia Lewis", "email": "amelia.lewis@icloud.com" },
{ "name": "Henry Walker", "email": "henry.walker@outlook.com" },
{ "name": "Evelyn Hall", "email": "evelyn.hall@gmail.com" },
{ "name": "Alexander Allen", "email": "alexander.allen@hotmail.com" },
{ "name": "Harper Young", "email": "harper.young@yahoo.com" },
{ "name": "Elijah King", "email": "elijah.king@gmail.com" },
{ "name": "Isabella Wright", "email": "isabella.wright@icloud.com" },
{ "name": "Sebastian Scott", "email": "sebastian.scott@aol.com" },
{ "name": "Avery Green", "email": "avery.green@yahoo.com" },
{ "name": "Jack Adams", "email": "jack.adams@gmail.com" },
{ "name": "Lily Nelson", "email": "lily.nelson@outlook.com" },
{ "name": "Matthew Baker", "email": "matthew.baker@hotmail.com" },
{ "name": "Scarlett Gonzalez", "email": "scarlett.gonzalez@yahoo.com" },
{ "name": "Joseph Carter", "email": "joseph.carter@gmail.com" },
{ "name": "Chloe Perez", "email": "chloe.perez@icloud.com" },
{ "name": "Samuel Turner", "email": "samuel.turner@hotmail.com" },
{ "name": "Zoey Collins", "email": "zoey.collins@yahoo.com" },
{ "name": "David Edwards", "email": "david.edwards@gmail.com" },
{ "name": "Ella Stewart", "email": "ella.stewart@aol.com" },
{ "name": "Gabriel Flores", "email": "gabriel.flores@outlook.com" },
{ "name": "Aria Morris", "email": "aria.morris@gmail.com" },
{ "name": "Nathaniel Rivera", "email": "nathaniel.rivera@icloud.com" },
{ "name": "Penelope Cooper", "email": "penelope.cooper@hotmail.com" },
{ "name": "Andrew Reed", "email": "andrew.reed@yahoo.com" },
{ "name": "Hannah Bailey", "email": "hannah.bailey@gmail.com" },
{ "name": "Christopher Bell", "email": "christopher.bell@outlook.com" },
{ "name": "Madison Murphy", "email": "madison.murphy@hotmail.com" },
{ "name": "Dylan Howard", "email": "dylan.howard@gmail.com" },
{ "name": "Layla Cox", "email": "layla.cox@yahoo.com" },
{ "name": "Ethan Ward", "email": "ethan.ward@icloud.com" },
{ "name": "Grace Brooks", "email": "grace.brooks@hotmail.com" },
{ "name": "Jackson Torres", "email": "jackson.torres@gmail.com" },
{ "name": "Victoria Sanders", "email": "victoria.sanders@outlook.com" }
]


In [ ]:
all_emails = pd.DataFrame(columns=["customer_id", "order_id", "products", "email", "customer_name", "email_address"])

for i in range(51):
    customer_id = customers["customer_id"].values[i]
    print(customer_id)
    customer_order_info = orders[orders["customer_id"] == customer_id][['order_id', 'customer_id', 'order_purchase_timestamp',
       'order_approved_at','order_estimated_delivery_date']]
    order_id = customer_order_info["order_id"].values.tolist()[0]
    order_time = customer_order_info["order_purchase_timestamp"].values.tolist()[0]
    customer_order_items = order_items[order_items["order_id"] == order_id]
   #  customer_payment_info = order_payment[order_payment["order_id"] == order_id]
    products = customer_order_items["product_category_name_english"].values.tolist()


    prompt = """ Write a realistic email to customer support from """ + customer_name_email[i]["name"] + """ (""" + customer_name_email[i]["email"] +""") asking about the status of their order.
                 Which was ordered at  """ + order_time + """. Do not include a phone number.
                 Order number:  """ + order_id
    
    email = query_open_ai(prompt)

    new_email = pd.DataFrame([[customer_id, order_id, products, email, customer_name_email[i]["name"], customer_name_email[i]["email"]]], 
                             columns=["customer_id", "order_id", "products", "email", "customer_name", "email_address"])
    all_emails = pd.concat([all_emails, new_email])

1be70cd84bb8fdc5c77a52054d6250f8
a8efde73814dc81ff114f0980fb6386f
46e7342ce409f2b5656f102f2ba9b9dc
65060b85a336b487c0fe7d42d64b3b99
73a8c93b316350e98f0d0804293f2c27
eb58af3f31ba5d587163397ba551e188
af6b5c7327d3dc4e1c212fa747706713
9babe15f3a8d05c3ae49e0e57faa9588
86d44d270b72592dbef78de27ec82d08
6b58b82f79f4f6d4b1663dbc9824ce15
adfd5b68ee720910253520bfcf6b3b1a
6fd5c621e78c434aa741e9ffb9c68250
b26b21a6be03433cf3052ee4b22a69c3
cacf236a67a1b018693b6e7309a3f670
84055878b8547a9f9944a8108146cf4f
9b684ecb3042a93d9476416aa979f289
cda5f27136ac1f174e1ba59d0bfea170
60be07ae31950ebb5bb4c2202837abc9
d3c3354f609a1a80b78fbab8d5435061
a10bf76efea959603e21cbfc13bbf221
64cd9338d3ef094dd406d660f738e7e3
cff21f454392b393d14858896b612102
1c2892b86921932bc4c4ac3045481688
3506351d6f6e36e9b381d2cbd40000b0
ed7abe2bf6dacb66b55c2bebf0d2adaf
a582b5b6122b059b61fd2151d0ed11fd
e92e2b7d6b397c76ae6dbbc5b351e9cf
431aa8450d30ae25badcaf4186f88281
9044dc01c6e9883f46c56c147c6c61f5
3e7b2479252ab3087413e44eb3354263
6e53638ba9

In [76]:
for email in all_emails["email"].reset_index(drop=True):
    print(email)
    print("")

Subject: Inquiry About Order Status - Order #9b1c817fd98473c0a0dab2a95b5979a6

Dear Customer Support Team,

I hope this message finds you well. I am writing to inquire about the status of my recent order placed on August 6, 2017. The order number is 9b1c817fd98473c0a0dab2a95b5979a6.

As it has been some time since I placed the order, I am eager to know the current status and estimated delivery date. I would appreciate any updates you could provide regarding this order.

Please let me know if there is any additional information you need from my side to facilitate this inquiry.

Thank you for your assistance.

Best regards,

John Smith
john.smith@gmail.com

Subject: Inquiry About Order Status - Order #d242f228c4dd03301794ded8f02dce54

Dear Customer Support,

I hope this message finds you well. I am writing to inquire about the status of my recent order placed on April 19, 2018, with the order number d242f228c4dd03301794ded8f02dce54. It has been some time since I placed the order, and I h

In [ ]:
# all_emails["category"] = "Order Status"

# all_emails[['customer_id', 'order_id', 'email', 'customer_name','email_address', 'category']].to_csv(data_folder + 'order_status_emails.csv')

In [10]:
all_emails = pd.read_csv(data_folder + 'order_status_emails.csv') 

In [12]:
all_emails.head()

,Unnamed: 0,customer_id,order_id,email,customer_name,email_address,category
0,0,1be70cd84bb8fdc5c77a52054d6250f8,9b1c817fd98473c0a0dab2a95b5979a6,Subject: Inquiry About Order Status - Order #9...,John Smith,john.smith@gmail.com,Order Status
1,0,a8efde73814dc81ff114f0980fb6386f,d242f228c4dd03301794ded8f02dce54,Subject: Inquiry About Order Status - Order #d...,Jane Doe,jane.doe@yahoo.com,Order Status
2,0,46e7342ce409f2b5656f102f2ba9b9dc,c67eb6f352d61cbfeec214480d5c6baf,Subject: Inquiry About Order Status - Order #c...,Alice Johnson,alice.johnson@outlook.com,Order Status
3,0,65060b85a336b487c0fe7d42d64b3b99,5e5c29987bbb83ed38c9b25a753f978f,Subject: Inquiry About Order Status - Order #5...,Robert Brown,robert.brown@gmail.com,Order Status
4,0,73a8c93b316350e98f0d0804293f2c27,db9564d23d5d0d213eb2a0b5ca279c2f,Subject: Inquiry About Order Status - Order #d...,Emily Davis,emily.davis@hotmail.com,Order Status


In [33]:
timestamps =[]
for index, row in all_emails.iterrows():
    customer_id = row["customer_id"]
    order_id = row["order_id"]
    email = row["email"]
    order_info = orders[(orders["customer_id"].astype(str) == str(customer_id)) & (orders["order_id"].astype(str) == str(order_id))]

    prompt = """ Determine when this email might have been sent. Based on the on this email: """ + email + """

                 And this order information  """ + order_info.to_json() + """. When might this email have been sent.  
                 Return only the timestamp in the format 'year-month-date hour:minute:second'. 
                 Do not return any additional information. Only return the timestamp.""" 
    
    time = query_open_ai(prompt)
    timestamps += [time]

In [36]:
all_emails["email_recieved"] = pd.DataFrame(timestamps)

In [38]:
all_emails.columns

Index(['Unnamed: 0', 'customer_id', 'order_id', 'email', 'customer_name',
       'email_address', 'category', 'email_recieved'],
      dtype='object')

In [ ]:
# all_emails[[ 'customer_id', 'order_id', 'email', 'customer_name',
#        'email_address', 'category', 'email_recieved']].to_csv(data_folder + 'order_status_emails.csv')